In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from PIL import Image
import pandas as pd



In [ ]:
def show_stats(df_):
    df = df_.copy()
    
    # Plotting the frequency of categories
    plt.figure(figsize=(10, 6))
    sns.countplot(y=df['meme_anxiety_categories'], order=df['meme_anxiety_categories'].value_counts().index)
    
    plt.title('Frequency of Meme Anxiety Categories')
    plt.xlabel('Count')
    plt.ylabel('Categories')
    plt.show()

In [ ]:
test_df = pd.read_csv('directory/test.csv')
train_df = pd.read_csv('directory/train.csv')



In [5]:
train_df.dropna(inplace = True)
test_df.dropna(inplace = True)

In [6]:
train_df.drop(columns='Unnamed: 0',inplace = True)
test_df.drop(columns='Unnamed: 0',inplace = True)

In [7]:

train_df = train_df[train_df['meme_anxiety_categories'] != 'Irritatbily']


In [8]:

test_df = test_df[test_df['meme_anxiety_categories'] != 'Irritatbily']


In [9]:
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
print(train_df.info())
print(test_df.info())

show_stats(train_df)
show_stats(test_df)

In [ ]:
test_ocr_array = torch.load('path to test ocr embeddings')
train_ocr_array = torch.load('path to train ocr embeddings')

test_gpt_array = torch.load('path to test gpt embeddings')
train_gpt_array = torch.load('path to train gpt embeddings')


print(train_gpt_array.shape)
print(train_ocr_array.shape) 
print(test_gpt_array.shape)
print(test_ocr_array.shape) 

In [12]:
test = torch.cat([test_ocr_array, test_gpt_array], dim=1)
test.shape

train = torch.cat([train_ocr_array, train_gpt_array], dim=1)
train.shape

torch.save(train, 'train_temp.pt')
torch.save(test, 'test_temp.pt')

In [ ]:
def get_image(test=None, train=None, index_=10):
    if test:
        image = f'path to anxiety test image/TE-{index_+1}.jpg'
    if train:
        image = f'path to anxiety train image/TR-{index_+1}.jpg'
    image = Image.open(image)
    image_resized = image.resize((350, 350))
    return image_resized

In [ ]:
test_n = 233
test_tensor = test[test_n]
print(test_df['triples'][test_n])
print(test_df['meme_anxiety_categories'][test_n])

get_image(test=True, index_ = test_n)

In [15]:
from filter import get_filtered

similarity = get_filtered(variable = train)

In [16]:
indices = similarity.get_filter_variable(test_tensor,
                              top_k = 1,
                              return_text=False)
print(indices)

[772]


In [ ]:
print(train_df['triples'][309])
print(train_df['meme_anxiety_categories'][309])

get_image(train = True, index_ = 309)

In [ ]:
from tqdm import tqdm
retrived_descs = []
retrived_labels = []
retrived_ocrs = []
indices_ = []
for n, desc in enumerate(tqdm(train_df['ocr_text'])):
    input_tensor = train[n]
    indices = similarity.get_filter_variable(input_tensor,
                              top_k = 4,
                              return_text=False)
    first_one = indices.pop(0)
    indices_.append(indices)

In [ ]:
train_df['indices'] = None
train_df['indices'] = indices_

train_df.to_csv('path to final dataframe for train.csv')

In [ ]:
from tqdm import tqdm
retrived_descs = []
retrived_labels = []
retrived_ocrs = []
indices_ = []
for n, desc in enumerate(tqdm(test_df['ocr_text'])):
    input_tensor = test[n]
    indices = similarity.get_filter_variable(input_tensor,
                              top_k = 3,
                              return_text=False)
    indices_.append(indices)

In [ ]:
test_df['indices'] = None
test_df['indices'] = indices_

test_df.to_csv('path to final dataframe for test.csv')